<h1><b>Transferencia de estilo con TensorHub</b></h1>

<author>Julio Waissman Vilanova</author>

<br/>

<a target="_blank" href="https://colab.research.google.com/github/juliowaissman/intro-rn/blob/main/transfer_img.ipynb">
<img src="https://i.ibb.co/2P3SLwK/colab.png" width=30pt />
<i>Para usar en Google Colab</i></a>

## Tranferencia de estilo

Tomado de [este ejemplo](https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization) y basado en el código del modelo en [magenta](https://github.com/tensorflow/magenta/tree/master/magenta/models/arbitrary_image_stylization) y la publicación:

[Explorar la estructura de un tiempo real, la red arbitraria estilización artística neuronal](https://arxiv.org/abs/1705.06830) . *Golnaz Ghiasi, Honglak Lee, Manjunath Kudlur, Vincent Dumoulin, Jonathon Shlens*, Actas de la Conferencia British Machine Vision (BMVC), 2017.

Empecemos importando las librerías que vamos a necesitar:

In [ ]:
# Para descargar las imagenes de estilo y contenido
import functools
import os

# Para graficar las imágenes
from matplotlib import gridspec
import matplotlib.pylab as plt

# Las que no pueden faltar
import numpy as np
import tensorflow as tf

# Para obtener los modelos preentrenados
import tensorflow_hub as hub

# Vamos viendo un poco de información
print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))

Vamos primero a definir algunas funciones para descargar las imágenes y para visualizarlas. 

Es importante aqui hablar sobre decoradores en python, si es que no los conocen. En este caso el decorador (de las herramientas de programación funcional de python) es para agregar **memoización** a la función. Esto es, que si la función ya se ejecuto una vez con esas entradas, guadre la salida en una memoria cache, para no tener que recalcularlo cada vez.


In [ ]:
def crop_center(image):
  """
  Recibe una imagen de (3, n, m) y regresa
  una imagen cuadrada de (3, min(n,m), min(n,m))
  centrada
  
  """
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape
  )
  
  return image

def show_n(images, titles=('',)):
  """
  Muestra una serie de imagenes en forma horizontal
  y les agrega los títulos correspondientes

  """
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()


@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
  """
  Carga y procesa una imagen. 

  Usa memoización (@functools.lru_cache) para no descargar la misma imagen muchas veces
  
  """
  # Descarga el archivo de la imagen en crudo
  image_path = tf.keras.utils.get_file(
      os.path.basename(image_url)[-128:], image_url
  )

  # Carga y procesa la imagen
  img = tf.io.decode_image(
      tf.io.read_file(image_path),
      channels=3, dtype=tf.float32
  )[tf.newaxis, ...] 

  # Recorta y escala la imagen 
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  
  return img



Vamos ahora a cargar unas imágenes

In [ ]:
content_image_url = 'https://proyectopuente.com.mx/wp-content/uploads/2021/03/Unison-2.jpg'
#style_image_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvjSkzPQVNjS5XOWmOwTdOdY_PaXB4T9KNW0cJIw3LIsL0zCZuhApTOfu7D0yD8dj_PhI&usqp=CAU'  
style_image_url = 'https://media.admagazine.com/photos/618a6b39b67f79aa891edc46/master/w_1600%2Cc_limit/63307.jpg'
output_image_size = 384 

# La imagen con el contenido, puede ser de un tamaño arbitrario.
content_img_size = (output_image_size, output_image_size)

# La imagen con el estilo. Puede ser de cualquier tamaño pero el modelo que vamos a usar
# está optimizado para imagenes de 256 x 256
style_img_size = (256, 256)  # Recomendado mantener en 256.

content_image = load_image(content_image_url, content_img_size)
style_image = load_image(style_image_url, style_img_size)
style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
show_n([content_image, style_image], ['Content image', 'Style image'])

Y ahora vamos a cargar el modelo preentrenado, el cual podemos revisar en la [página de TensorFlow Hub de este modelo](https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2)

In [ ]:
hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

Y simplemente lo ejecutamos y vemos el resultado

In [ ]:
# Stylize content image with given style image.
# This is pretty fast within a few milliseconds on a GPU.

outputs = hub_module(
    tf.constant(content_image), 
    tf.constant(style_image)
)
stylized_image = outputs[0]

# Visualize input images and the generated stylized image.

show_n(
    [content_image, style_image, stylized_image], 
    titles=['Original content image', 'Style image', 'Stylized image']
)